In [1]:
import pandas as pd

In [2]:
DATA_FOLDER = '../../data/'

In [3]:
organizations_df = pd.read_parquet(f'{DATA_FOLDER}organizations.parquet')

In [4]:
organizations_df

,organization
0,Biomedical Informatics
1,Shandong University
2,Tianjin University
3,Tsinghua University
4,City University of Hong Kong
...,...
105,Data Scientist
106,Amazon.com
107,"Biostatistics, University of Michigan"
108,Computer Science and Engineeing


In [7]:
def search_papers(query, limit=5):
    search_query = scholarly.search_pubs(query)
    papers = []

    for _ in range(limit):
        try:
            paper = next(search_query)
                        
            papers.append({
                "title": paper.get("bib", {}).get("title"),
                "abstract": paper.get("bib", {}).get("abstract"),
                "year": paper.get("bib", {}).get("pub_year"),
                "url": paper.get("eprint_url", "Unknown"),
                "author_id": paper.get("author_id", [])
            })
        except StopIteration:
            break
        except Exception as e:
            print(f"Error processing paper : {e}")
            continue

    return papers

In [27]:
def search_authors(papers):
    author_ids = list()
    authors = list()
    for paper in papers:
        print
        if len(author_ids) > 10:
            break
        if "author_id" in paper:
            for author_id in paper["author_id"]:
                if author_id: # check the author_id is not empty
                    #time.sleep(2)  # Introduce delay
                    if author_id not in author_ids: # avoids duplicate searches                        
                        try:
                            print(f'Searching for author with id {author_id}...')
                            author = scholarly.search_author_id(author_id)
                            #author = scholarly.fill(scholarly.search_author_id(author_id), sections=['basics'])
                            authors.append({
                                "author_id": author_id,
                                "name": author.get("name"),
                                "affiliation": author.get("affiliation", "Unknown")
                            })
                            author_ids.append(author_id)
                        except Exception as e:
                            print(f"Error processing author with id {author_id}: {e}")
    return authors

# Lookup and store geolocations

In [14]:
import geopandas as gpd
import folium
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="lncRNA-map")

# https://geopy.readthedocs.io/en/stable/#geopy.geocoders.options.default_timeout
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)

In [15]:
from tqdm import tqdm

In [16]:
tqdm.pandas()
organizations_df['location'] = organizations_df['organization'].progress_apply(geocode)

100%|████████████████████████████████████████████████████████████████████████████████████| 110/110 [03:38<00:00,  1.98s/it]


In [34]:
organizations_df.to_pickle(f'{DATA_FOLDER}organizations_with_location.pickle')